In [132]:
# This program will combine and clean the high school performance data, as well as add location information
# Dependencies and Setup
import pandas as pd
import numpy as np

# Files to Load
DOE_2013_2014_report = "Resources/2013_-_2014_DOE_High_School_Performance-Directory.csv"
DOE_2014_2015_report = "Resources/2014-2015_DOE_High_School_Performance-Directory.csv"
DOE_2016_report = "Resources/2016_DOE_High_School_Performance__Directory.csv"
school_safety_report = "Resources/2010_-_2016_School_Safety_Report.csv"
precinct_key_csv = "Resources/Precinct_Key.xlsx"

# Read Purchasing File and store into Pandas data frame
DOE_2013_2014 = pd.read_csv(DOE_2013_2014_report)
DOE_2014_2015 = pd.read_csv(DOE_2014_2015_report)
DOE_2016 = pd.read_csv(DOE_2016_report)
school_safety = pd.read_csv(school_safety_report)
precinct_key = pd.read_excel(precinct_key_csv)

In [133]:
# data cleaning

# select needed columns
DOE_2013_2014 = DOE_2013_2014[["DBN", "graduation 2010-11", "college enroll 2010-11", "graduation 2011-12", "college enroll 2011-12"]]
DOE_2014_2015 = DOE_2014_2015[["dbn", "graduation_rate_2013", "college_career_rate_2013"]]
DOE_2016 = DOE_2016[["dbn", "graduation_rate_2014", "college_career_rate_2014"]]

# rename column headers
DOE_2013_2014 = DOE_2013_2014.rename(index=str, columns={"graduation 2010-11": "graduation_rate_2011",
                                                         "college enroll 2010-11": "college_career_rate_2011",
                                                         "graduation 2011-12": "graduation_rate_2012",
                                                         "college enroll 2011-12": "college_career_rate_2012"})
DOE_2014_2015 = DOE_2014_2015.rename(index=str, columns={"dbn": "DBN"})
DOE_2016 = DOE_2016.rename(index=str, columns={"dbn": "DBN"})

# drop rows with missing data
DOE_2013_2014 = DOE_2013_2014.dropna(how='any')
DOE_2014_2015 = DOE_2014_2015.dropna(how='any')
DOE_2016 = DOE_2016.dropna(how='any')

In [135]:
# merge the 3 datasets
merge0 = pd.merge(DOE_2013_2014, DOE_2014_2015, on="DBN")
merge1 = pd.merge(merge0, DOE_2016, on="DBN")

In [141]:
# bring in geographic data
geo_safety_data = school_safety[["School Year", "DBN", "Community Board", "Council District ", "Census Tract", "NTA"]]
geo_safety_data = geo_safety_data.loc[geo_safety_data["School Year"] == "2013-14"]
geo_safety_data = geo_safety_data.drop(columns="School Year")

merge2 = pd.merge(merge1, geo_safety_data, on="DBN")

# clean NTA column (remove trailing white space)
merge2["NTA"] = merge2["NTA"].str.strip()

# bring in precinct number
precinct_key = precinct_key[["DBN", "Precinct"]]
merge3 = pd.merge(merge2, precinct_key, on="DBN")

In [143]:
# export to excel file
merge3.to_excel("Resources/Combined_HS_Performance_Data.xlsx", index=False)